# New Scraper

In [1]:
import bs4 as bs
import requests
import pandas as pd

In [2]:
'HI'.lower()

'hi'

In [3]:
import requests

url = 'https://www.worldometers.info/coronavirus/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)
df = dfs[0]

df = df[['Country,Other','TotalDeaths']]

df = df.transpose()

df.columns = df.loc['Country,Other']

df = df.drop('Country,Other')

In [4]:
import datetime

str(datetime.datetime.now())

'2020-03-30 20:24:01.412446'

In [5]:
import requests
import datetime

def death_update():

    url = 'https://www.worldometers.info/coronavirus/'

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)
    df = dfs[0]

    df = df[['Country,Other','TotalDeaths']]
    
    df.columns = ['Date','Deaths']

    df = df.transpose()

    df.columns = df.loc['Date']

    df = df.drop('Date')
    
    df.index = [datetime.datetime.now().date()]
    
    df.set_index
    
    return(df)

In [6]:
df = death_update()[['Sweden','Denmark','Norway','Finland','Iceland']]

In [7]:
df

Date,Sweden,Denmark,Norway,Finland,Iceland
2020-03-30,146,77,32,13,2


In [8]:
import html5lib

In [9]:
pd.read_html('https://www.worldometers.info/coronavirus/')

HTTPError: HTTP Error 403: Forbidden

In [ ]:
def scrape_country_deaths(country):
    contry = country.lower()
    source = requests.get('https://www.worldometers.info/coronavirus/country/{}/'.format(country)).content
    soup = bs.BeautifulSoup(source, features='html.parser')
    return(int(soup.find_all('div',class_='maincounter-number')[1].span.text))

In [ ]:
scrape_country_deaths('Icleand')

In [ ]:
    url = 'https://pomber.github.io/covid19/timeseries.json'
    countries = ['Sweden', 'Denmark', 'Norway', 'Finland', 'Iceland','Italy','Spain','France','Germany']
    df = pd.read_json(url)

In [45]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import json


def get_data():
    import requests
    url = 'https://pomber.github.io/covid19/timeseries.json'
    countries = ['Sweden', 'Denmark', 'Norway', 'Finland', 'Iceland','Italy','Spain','France','Germany','US','United Kingdom','Poland','Switzerland','Slovenia','Slovakia','Portugal','Netherlands','Japan','Korea, South','Ireland','Estonia','Czechia','China','Canada','Belgium','Austria','Australia']
    df = pd.read_json(url)[countries]

    n_rows = df.shape[0]
    df['Sweden'][0]
    dates = []
    for i in range(n_rows):
        dates.append(df['Sweden'][i]['date'])

    df['Date'] = dates
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    
    df = df.rename(columns={'US':'United States','Korea, South':'South Korea', 'Czechia':'Czech Republic'})

    df_deaths = pd.DataFrame(index=df.index)
    for col in df.columns:
        df_deaths[col] = [c.get('deaths') for c in df[col]]

    # Start from March 10 before first deaths
    #df_deaths = df_deaths['2020-03-10':]
    # Fix faulty Iceland data
    df_deaths.loc['2020-03-15', 'Iceland'] = 0
    df_deaths.loc['2020-03-20', 'Iceland'] = 1

    df_pop = pd.read_html(
        'https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)')[3]

    df_pop = df_pop[['Country or area', 'Population(1 July 2019)']]

    df_pop['Country or area'] = df_pop['Country or area'].str.replace(
        '\[.*\]', '')
    df_pop = df_pop.pivot_table(
        columns='Country or area', values='Population(1 July 2019)')[df.columns]
    df_pop = df_pop / 1000000
    df_deaths_per_mn = pd.DataFrame(index=df_deaths.index)

    for col in df_deaths.columns:
        df_deaths_per_mn[col] = df_deaths[col] / df_pop[col].values

    df_deaths_1 = df_deaths[df_deaths != 0]
    # Remove all dates with zero deaths
    df_deaths_1 = df_deaths_1.apply(lambda x: pd.Series(x.dropna().values))

    # Add zero to first day
    df_deaths_1 = pd.concat([pd.DataFrame(np.zeros((1, df_deaths_1.shape[1])),
                                          columns=df_deaths_1.columns), df_deaths_1], axis=0, ignore_index=True)

    # deaths per mn inhabitants since first death
    df_deaths_per_mn_1 = pd.DataFrame(index=df_deaths_1.index)
    for col in df_deaths.columns:
        df_deaths_per_mn_1[col] = df_deaths_1[col] / df_pop[col].values

    date = df_deaths.index[-1].to_pydatetime()+timedelta(days=1)
    obj = {'df_deaths': df_deaths.to_json(), 'df_deaths_per_mn': df_deaths_per_mn.to_json(),
           'df_deaths_1': df_deaths_1.to_json(), 'df_deaths_per_mn_1': df_deaths_per_mn_1.to_json(),
           'date': datetime.timestamp(date)}
    #return json.dumps(obj)
    return(df_deaths)

In [46]:
df = get_data()

In [48]:
import flag

flag.flag('SE')

'🇸🇪'

In [49]:
df_flags = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2')[2]

In [50]:
df_flags

,Code,Country name (using title case),Year,ccTLD,ISO 3166-2,Notes
0,AD,Andorra,1974,.ad,ISO 3166-2:AD,NaN
1,AE,United Arab Emirates,1974,.ae,ISO 3166-2:AE,NaN
2,AF,Afghanistan,1974,.af,ISO 3166-2:AF,NaN
3,AG,Antigua and Barbuda,1974,.ag,ISO 3166-2:AG,NaN
4,AI,Anguilla,1985,.ai,ISO 3166-2:AI,AI previously represented French Afars and Issas
...,...,...,...,...,...,...
244,YE,Yemen,1974,.ye,ISO 3166-2:YE,"Previous ISO country name: Yemen, Republic of ..."
245,YT,Mayotte,1993,.yt,ISO 3166-2:YT,NaN
246,ZA,South Africa,1974,.za,ISO 3166-2:ZA,Code taken from name in Dutch: Zuid-Afrika
247,ZM,Zambia,1974,.zm,ISO 3166-2:ZM,NaN


In [51]:
df_flags = df_flags[['Code', 'Country name (using title case)']]
df_flags.columns = ['Code','Country']

In [52]:
countries = list(df.columns)
df_flags[df_flags['Country'].isin(countries)]

,Code,Country
11,AT,Austria
12,AU,Australia
19,BE,Belgium
37,CA,Canada
42,CH,Switzerland
47,CN,China
56,DE,Germany
58,DK,Denmark
63,EE,Estonia
67,ES,Spain
